In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [2]:
THRESH_SPO2_LOW = 92
THRESH_SPO2_CRITICAL = 88
THRESH_SHOCK_INDEX_WARNING = 0.9
THRESH_SHOCK_INDEX_CRITICAL = 1.0
THRESH_MAP_LOW = 65  # Mean Arterial Pressure threshold for shock
THRESH_LACTATE_HIGH = 2.0  # mmol/L for tissue hypoperfusion
THRESH_LACTATE_CRITICAL = 4.0
THRESH_UO_LOW = 0.5  # mL/kg/hr (oliguria)

# Trend Analysis
TREND_WINDOW = 6  # Number of readings for short-term trend analysis

# AGE_THRESHOLDS (kept same as user-provided)
AGE_THRESHOLDS = {
    'neonate': {
        'rr_low': 30, 'rr_normal': 40, 'rr_high': 60,
        'hr_low': 100, 'hr_normal': 140, 'hr_high': 160,
        'sbp_low': 60, 'sbp_normal': 70, 'sbp_high': 90,
        'temp_low': 36.0, 'temp_normal': 37.2, 'temp_high': 38.0
    },
    'infant': {
        'rr_low': 24, 'rr_normal': 30, 'rr_high': 40,
        'hr_low': 80, 'hr_normal': 120, 'hr_high': 140,
        'sbp_low': 70, 'sbp_normal': 85, 'sbp_high': 100,
        'temp_low': 36.0, 'temp_normal': 37.2, 'temp_high': 38.0
    },
    'child': {
        'rr_low': 16, 'rr_normal': 20, 'rr_high': 30,
        'hr_low': 70, 'hr_normal': 90, 'hr_high': 110,
        'sbp_low': 80, 'sbp_normal': 95, 'sbp_high': 110,
        'temp_low': 36.0, 'temp_normal': 37.0, 'temp_high': 38.0
    },
    'adolescent': {
        'rr_low': 12, 'rr_normal': 16, 'rr_high': 20,
        'hr_low': 60, 'hr_normal': 75, 'hr_high': 100,
        'sbp_low': 90, 'sbp_normal': 105, 'sbp_high': 120,
        'temp_low': 35.8, 'temp_normal': 36.8, 'temp_high': 37.8
    },
    'adult': {
        'rr_low': 12, 'rr_normal': 16, 'rr_high': 20,
        'hr_low': 60, 'hr_normal': 80, 'hr_high': 100,
        'sbp_low': 90, 'sbp_normal': 115, 'sbp_high': 130,
        'temp_low': 35.5, 'temp_normal': 36.8, 'temp_high': 38.0
    },
    'geriatric': {
        'rr_low': 12, 'rr_normal': 16, 'rr_high': 24,
        'hr_low': 55, 'hr_normal': 70, 'hr_high': 90,
        'sbp_low': 90, 'sbp_normal': 125, 'sbp_high': 140,
        'temp_low': 35.5, 'temp_normal': 36.5, 'temp_high': 37.5
    }
}

In [3]:
def assign_age_category(df):
    df = df.copy()
    def _categorize(age):
        if age <= 0.083: return 'neonate'
        elif age <= 1:   return 'infant'
        elif age < 5:    return 'child'
        elif age < 13:   return 'adolescent'
        elif age < 65:   return 'adult'
        else:            return 'geriatric'
    if 'age' in df.columns:
        df['age_category'] = df['age'].apply(_categorize)
    else:
        df['age_category'] = 'adult'
    return df

In [4]:
def apply_vital_range_flags(df):
    df = df.copy()
    df = assign_age_category(df)

    # SpO₂ flags
    df['flag_spo2_low'] = df['spo2'] < THRESH_SPO2_LOW
    df['flag_spo2_critical'] = df['spo2'] < THRESH_SPO2_CRITICAL

    # Temperature flags
    df['flag_temp_high'] = df.apply(lambda row: row['temperature'] >= AGE_THRESHOLDS[row['age_category']]['temp_high'], axis=1)
    df['flag_temp_low'] = df.apply(lambda row: row['temperature'] < AGE_THRESHOLDS[row['age_category']]['temp_low'], axis=1)

    # Respiratory Rate flags
    df['flag_rr_low'] = df.apply(lambda row: row['resp_rate'] < AGE_THRESHOLDS[row['age_category']]['rr_low'], axis=1)
    df['flag_rr_high'] = df.apply(lambda row: row['resp_rate'] >= AGE_THRESHOLDS[row['age_category']]['rr_high'], axis=1)

    # Heart Rate flags
    df['flag_hr_low'] = df.apply(lambda row: row['heart_rate'] < AGE_THRESHOLDS[row['age_category']]['hr_low'], axis=1)
    df['flag_hr_high'] = df.apply(lambda row: row['heart_rate'] >= AGE_THRESHOLDS[row['age_category']]['hr_high'], axis=1)

    # Shock Index
    df['shock_index'] = df['heart_rate'] / np.clip(df['sbp'], a_min=1, a_max=None)
    df['flag_si_warning'] = df['shock_index'] >= THRESH_SHOCK_INDEX_WARNING
    df['flag_si_critical'] = df['shock_index'] >= THRESH_SHOCK_INDEX_CRITICAL

    # Blood Pressure flags (sbp/dbp)
    df['flag_sbp_low'] = df.apply(lambda row: row['sbp'] < AGE_THRESHOLDS[row['age_category']]['sbp_low'], axis=1)
    df['flag_sbp_high'] = df.apply(lambda row: row['sbp'] >= AGE_THRESHOLDS[row['age_category']]['sbp_high'], axis=1)
    df['flag_dbp_low'] = df.apply(lambda row: row['dbp'] < (AGE_THRESHOLDS[row['age_category']]['sbp_low'] * 0.6), axis=1)
    df['flag_dbp_high'] = df.apply(lambda row: row['dbp'] >= (AGE_THRESHOLDS[row['age_category']]['sbp_high'] * 0.6), axis=1)

    return df

In [5]:
def compute_recent_trends_delta(df, trend_window=TREND_WINDOW):
    df = df.copy().sort_values("timestamp").reset_index(drop=True)
    if 'age_category' not in df.columns:
        df = assign_age_category(df)
    trends = {}
    recent = df.tail(trend_window)
    if recent.empty:
        return trends
    age_group = recent['age_category'].iloc[-1]
    thresholds = AGE_THRESHOLDS[age_group]

    vital_map = {
        'resp_rate': ('rr_low', 'rr_normal', 'rr_high'),
        'heart_rate': ('hr_low', 'hr_normal', 'hr_high'),
        'sbp': ('sbp_low', 'sbp_normal', 'sbp_high'),
        'temperature': ('temp_low', 'temp_normal', 'temp_high'),
        'spo2': (None, None, None)
    }

    for vital in ['resp_rate', 'heart_rate', 'sbp', 'temperature', 'spo2']:
        if vital not in recent.columns or recent[vital].isnull().all():
            continue
        y = recent[vital].dropna().values
        if len(y) < 2:
            continue
        avg_delta = np.mean(np.diff(y))
        latest = y[-1]
        trends[f"{vital}_trend"] = round(avg_delta, 3)

        if vital == 'spo2':
            if latest < THRESH_SPO2_LOW:
                if avg_delta > 0:
                    flag = "Still abnormal — but improving"
                elif avg_delta < 0:
                    flag = "Abnormal and worsening"
                else:
                    flag = "Abnormal and flat"
            else:
                if avg_delta < 0:
                    flag = "Normal but deteriorating"
                else:
                    flag = "Normal and stable"
        else:
            low_key, norm_key, high_key = vital_map[vital]
            low = thresholds[low_key]
            normal = thresholds[norm_key]
            high = thresholds[high_key]
            if latest < low or latest > high:
                if (latest > high and avg_delta < 0) or (latest < low and avg_delta > 0):
                    flag = "Still abnormal — but improving"
                else:
                    flag = "Abnormal and worsening"
            else:
                if avg_delta < 0:
                    flag = "Normal but deteriorating"
                else:
                    flag = "Normal and stable"
        trends[f"{vital}_trend_flag"] = flag

    # Shock Index trend
    if all(col in recent.columns for col in ['heart_rate', 'sbp']):
        hr = recent['heart_rate'].values
        sbp = np.clip(recent['sbp'].values, a_min=1, a_max=None)
        si = hr / sbp
        if len(si) >= 2:
            avg_si_delta = np.mean(np.diff(si))
            trends['shock_index_trend'] = round(avg_si_delta, 3)
            latest_si = si[-1]
            if latest_si >= THRESH_SHOCK_INDEX_CRITICAL:
                flag = "Shock Index critical — improving" if avg_si_delta < 0 else "Shock Index critical — worsening"
            else:
                flag = "Normal but improving" if avg_si_delta < 0 else "Normal but rising"
            trends['shock_index_trend_flag'] = flag
    return trends


In [6]:
def consecutive_true(series: pd.Series, min_minutes: int) -> bool:
    """
    Check if there are at least `min_minutes` consecutive True values in a boolean Series.
    """
    max_run = run = 0
    for val in series:
        if val:
            run += 1
            max_run = max(max_run, run)
        else:
            run = 0
    return max_run >= min_minutes

In [7]:
def detect_dird(min_df: pd.DataFrame, cfg: dict) -> dict:
    """
    Detect Drug-Induced Respiratory Depression (DIRD).
    Combines event detection + flag assignment for integration into mega pipeline.
    
    Criteria:
      • RR ≤ cfg['resp_depress']['rr_low'] for ≥ cfg['resp_depress']['min_minutes_low_rr'] (while at rest)
      • SpO₂ ≤ cfg['thresholds']['spo2_low'] (sustained via rolling window)
      • No fever / major SBP changes (not enforced in detection, but evidence recorded)
    """
    at_rest = min_df.get('at_rest')
    if at_rest is None:
        # naive fallback: consider night (8pm–7am) as rest
        at_rest = (~min_df.index.to_series().dt.hour.between(7, 20))

    # --- Low RR sustained event ---
    rr_low = (min_df['rr'] <= cfg['resp_depress']['rr_low']) & at_rest
    sustained_rr_low = consecutive_true(rr_low, cfg['resp_depress']['min_minutes_low_rr'])

    # --- SpO₂ sustained low ---
    spo2_low = (
        (min_df['spo2'] <= cfg['thresholds']['spo2_low'])
        .rolling(5, min_periods=5).mean().gt(0.6).any()
    )

    # --- Flags for integration ---
    flag_rr_low = bool(sustained_rr_low)
    flag_spo2_low = bool(spo2_low)
    flag_dird = flag_rr_low and flag_spo2_low

    # --- Build output for mega pipeline ---
    result = {
        "flag_dird": flag_dird,
        "flag_dird_rr_low": flag_rr_low,
        "flag_dird_spo2_low": flag_spo2_low,
        "label": (
            "Possible opioid/sedative respiratory depression — "
            "slow breathing with low oxygen. Assess sedation, "
            "consider naloxone; close monitoring."
            if flag_dird else None
        ),
        "evidence": {
            "rr≤10_sustained": flag_rr_low,
            "spo2≤92%": flag_spo2_low,
            "fever_any": bool((min_df['temperature'] >= 38).any()),
            "sbp_drop_any": bool((min_df['sbp'].diff().le(-20)).any())
        }
    }

    return result

In [8]:
rng = pd.date_range("2025-09-17 00:00:00", periods=60, freq="min")  # 60 minutes, 1-min resolution

# Simulate vitals
data = {
    "timestamp": rng,
    "rr": [16]*10 + [9]*20 + [16]*30,  # Normal RR, then prolonged low RR, then recovery
    "spo2": [96]*15 + [90]*15 + [95]*30,  # Drop in SpO₂ during RR depression
    "temperature": [37]*60,  # No fever
    "sbp": [120]*60,  # Stable SBP
    "age_category": ["adult"]*60
}

df = pd.DataFrame(data)

# Config thresholds (simplified for test)
cfg = {
    "resp_depress": {"rr_low": 10, "min_minutes_low_rr": 10},
    "thresholds": {"spo2_low": 92},
}
result = detect_dird(df.set_index('timestamp'), cfg)
print(result)

{'flag_dird': True, 'flag_dird_rr_low': True, 'flag_dird_spo2_low': True, 'label': 'Possible opioid/sedative respiratory depression — slow breathing with low oxygen. Assess sedation, consider naloxone; close monitoring.', 'evidence': {'rr≤10_sustained': True, 'spo2≤92%': True, 'fever_any': False, 'sbp_drop_any': False}}
